<div style ="border: blue solid 5px; font-family: Agency FB;	"><center>
<a href="#" ></a> 
    <h1> <font color="blue">Création Base SIREN</font> </h1> 
<a href="#" ></a>     
</center>
</div>

Dans ce notebook nous récupérons les siren présents dans la base Ademe afin de télécharger les données SIREN. Le site de l'INSEE (https://www.sirene.fr/sirene/public/creation-fichier) permet le téléchargement des données de 500 siren, raison pour laquelle nous subdivisons notre fichier en 4 sous listes.

## Table des matières 

* [Chargement des données Ademe](#1-chargement-des-données-ademe)
* [Extraction des listes de SIREN](#2-extraction-des-listes-de-siren)
* [Selection des variables dans SIREN](#3-selection-des-variables)
* [Enrichissement](#4-enrichissement)
* [Sauvegarde de la base](#sauvegarde-de-la-base-siren-finale)


## 1. Chargement des données Ademe

In [64]:
import pandas as pd
import numpy as np
import os 
from string import ascii_letters

import warnings
warnings.filterwarnings("ignore")

In [65]:
os.chdir("D:/01_ENSAI/Sem_2/Projet_Statistique/projet-risque-de-transition")
bilan_ges = pd.read_csv('bdd/Ademe/Ademe.csv')


In [66]:
bilan_ges.shape

(1860, 11)

## 2. Extraction des listes de SIREN

Comme le quota de nombre de SIREN défini par l'INSEE est de 500, nous avons réparti les SIREN dans 5 fichiers.

In [67]:
## Recupération des SIREN et répartition sur 5 fichiers
list_file = ["siren1", "siren2", "siren3", "siren4"]
first = 0
for file in list_file:
   siren_file = open("bdd/Code_Siren/{}.txt".format(file), "w")
   last = first + 465
         
   for i in list(bilan_ges.legal_unit_id)[first:last]:
      i = str(i)
      if len(i)<8:
         i = "00" + i
      if len(i)<9:
         i = "0" + i
      if len(i) !=9:
         print(file, i)
         break
      siren_file.write(str(i) + "\n")

   siren_file.close()
   first = last

On récupère chaque base SIREN et on les combine

In [68]:
## Append des différentes base
all_siren_import = pd.read_csv("bdd/BaseSiren/siren1.csv", low_memory=False)
for file in list_file[1:]:
   siren = pd.read_csv("bdd/BaseSiren/{}.csv".format(file), low_memory=False)
   all_siren_import = all_siren_import.append(siren, ignore_index=True)

In [69]:
all_siren = all_siren_import.copy()

In [71]:
verif_siren = bilan_ges.merge(all_siren, how='left', left_on='legal_unit_id', right_on='siren', validate='1:m')
siren_missing = verif_siren.loc[verif_siren.siren.isna(), 'legal_unit_id'].values

siren_missing



array([776986546, 757166730, 748801906, 720001437, 702034004, 699625013,
       573185684, 444571000, 424632759, 421110980, 403372800, 325446930,
       241647920, 194569100, 194469960,  45536234], dtype=int64)

In [72]:
## Recupération des siren manquants
siren_file = open("bdd/Code_Siren/siren0.txt", "w")
for i in siren_missing:
   i = str(i)
   if len(i)<8:
      i = "00" + i
   if len(i)<9:
      i = "0" + i
   if len(i)!=9:
      print(i)
      break
   siren_file.write(str(i) + "\n")
siren_file.close()

La base SIREN contient désormais les codes SIREN et les codes APE. Nous allons faire une jointure 
avec la base BilanGes afin d'identifier la filière de chaque entreprise

## 3. Selection des variables

Pour le moment les variables retenues sont choisies par lecture des métadonnées. Une analyse plus fine sera effectuée après fusion des bases de données

In [74]:
## Recupération des variables utiles
list_keep_siren = [
                  "siren", 
                  "siret",
                  "activitePrincipaleUniteLegale", 
                  "economieSocialeSolidaireUniteLegale",
                  "caractereEmployeurUniteLegale", 
                  "trancheEffectifsUniteLegale",
                  "anneeEffectifsEtablissement",
                  "categorieEntreprise", 
                  'codePostalEtablissement', 
                  'codeCommuneEtablissement',
                  'dateCreationUniteLegale',
                  'categorieJuridiqueUniteLegale',
                  'etablissementSiege',
                  'etatAdministratifUniteLegale',
                  ]
all_siren = all_siren[list_keep_siren]
all_siren.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217536 entries, 0 to 217535
Data columns (total 14 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   siren                                217536 non-null  int64  
 1   siret                                217536 non-null  int64  
 2   activitePrincipaleUniteLegale        217536 non-null  object 
 3   economieSocialeSolidaireUniteLegale  214346 non-null  object 
 4   caractereEmployeurUniteLegale        217536 non-null  object 
 5   trancheEffectifsUniteLegale          217004 non-null  object 
 6   anneeEffectifsEtablissement          73817 non-null   float64
 7   categorieEntreprise                  213464 non-null  object 
 8   codePostalEtablissement              217498 non-null  float64
 9   codeCommuneEtablissement             217498 non-null  object 
 10  dateCreationUniteLegale              217536 non-null  object 
 11  categorieJuri

## 4. Enrichissement

**Création des code APE niveau 1**

In [75]:
## Code APE à deux chiffres
all_siren['APE_2_code'] = [int(col[0:2]) for col in all_siren.activitePrincipaleUniteLegale]
## Assignation des codes aux sections
conditions = [
   (all_siren.APE_2_code <=3),
   (all_siren.APE_2_code >3) & (all_siren.APE_2_code <= 9),
   (all_siren.APE_2_code >9) & (all_siren.APE_2_code <= 33),
   (all_siren.APE_2_code ==35),
   (all_siren.APE_2_code >35) & (all_siren.APE_2_code <= 39),
   (all_siren.APE_2_code >39) & (all_siren.APE_2_code <= 43),
   (all_siren.APE_2_code >43) & (all_siren.APE_2_code <= 47),
   (all_siren.APE_2_code >47) & (all_siren.APE_2_code <= 53),
   (all_siren.APE_2_code >53) & (all_siren.APE_2_code <= 56),
   (all_siren.APE_2_code >56) & (all_siren.APE_2_code <= 63),
   (all_siren.APE_2_code >63) & (all_siren.APE_2_code <= 66),
   (all_siren.APE_2_code == 68),
   (all_siren.APE_2_code >68) & (all_siren.APE_2_code <= 75),
   (all_siren.APE_2_code >75) & (all_siren.APE_2_code <= 82),
   (all_siren.APE_2_code == 84),
   (all_siren.APE_2_code ==85),
   (all_siren.APE_2_code >85) & (all_siren.APE_2_code <= 88),
   (all_siren.APE_2_code >88) & (all_siren.APE_2_code <= 93),
   (all_siren.APE_2_code >93) & (all_siren.APE_2_code <= 96),
   (all_siren.APE_2_code >96) & (all_siren.APE_2_code <= 98),
   (all_siren.APE_2_code == 99)
]

## Section correspondant
Section_APE = [letter for letter in ascii_letters.upper()[0:21]]

In [76]:
all_siren['APE_niv_1'] = np.select(conditions, Section_APE)

In [77]:
all_siren[['activitePrincipaleUniteLegale', 'APE_niv_1']]

,activitePrincipaleUniteLegale,APE_niv_1
0,10.82Z,C
1,10.82Z,C
2,10.82Z,C
3,10.82Z,C
4,10.82Z,C
...,...,...
217531,49.41B,H
217532,49.41B,H
217533,49.41B,H
217534,49.41B,H


**Nombre de SIRET par SIREN**

In [78]:
count_siret = all_siren.groupby(["siren"])["siret"].count().reset_index(name="siret_count")
all_siren = all_siren.merge(count_siret, how = "left", left_on='siren', right_on='siren', validate='m:1')


In [79]:
all_siren.loc[all_siren.siren == 380129866, ['siren', 'siret_count']]

,siren,siret_count
114250,380129866,4421
114251,380129866,4421
114252,380129866,4421
114253,380129866,4421
114254,380129866,4421
...,...,...
118666,380129866,4421
118667,380129866,4421
118668,380129866,4421
118669,380129866,4421


**Taux de valeurs manquantes**

In [80]:
taux_na = all_siren.isna().sum()/all_siren.shape[0]
taux_na.values
var =list(all_siren.columns)
dtype = all_siren.dtypes.values
data = {
         'Variables' : var,
         'Dtype': dtype,
         'Taux_na': taux_na.values
               
      }
recap_na = pd.DataFrame(data, index = range(len(var)))
recap_na.sort_values(by = 'Taux_na', ascending=False, inplace= True)
recap_na

,Variables,Dtype,Taux_na
6,anneeEffectifsEtablissement,float64,0.660668
7,categorieEntreprise,object,0.018719
3,economieSocialeSolidaireUniteLegale,object,0.014664
5,trancheEffectifsUniteLegale,object,0.002446
8,codePostalEtablissement,float64,0.000175
9,codeCommuneEtablissement,object,0.000175
11,categorieJuridiqueUniteLegale,int64,0.000000
15,APE_niv_1,object,0.000000
14,APE_2_code,int64,0.000000
13,etatAdministratifUniteLegale,object,0.000000


**Retenons uniquement les établissements sièges**

In [81]:
all_siren.etablissementSiege.value_counts()

False    215692
True       1844
Name: etablissementSiege, dtype: int64

In [82]:
siren_df = all_siren[all_siren.etablissementSiege]
siren_df

,siren,siret,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,caractereEmployeurUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsEtablissement,categorieEntreprise,codePostalEtablissement,codeCommuneEtablissement,dateCreationUniteLegale,categorieJuridiqueUniteLegale,etablissementSiege,etatAdministratifUniteLegale,APE_2_code,APE_niv_1,siret_count
6,602018897,60201889700168,10.82Z,N,O,32,NaN,ETI,76130.0,76451,1960-01-01,5599,True,A,10,C,9
9,602020737,60202073700022,33.13Z,N,O,41,2019.0,GE,31700.0,31069,1900-01-01,5710,True,A,33,C,12
21,602025538,60202553800078,45.11Z,N,O,00,2019.0,GE,2600.0,02810,1960-01-01,5710,True,A,45,G,22
53,602039299,60203929900113,71.12B,N,O,32,NaN,GE,78180.0,78423,1960-01-01,5710,True,A,71,M,12
163,602044638,60204463801139,78.20Z,N,O,22,NaN,GE,94000.0,94028,1960-01-01,5710,True,A,78,N,137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217429,357200054,35720005400017,10.62Z,N,O,51,2019.0,GE,62136.0,62502,1957-01-01,5599,True,A,10,C,12
217441,357801109,35780110900020,27.11Z,N,O,32,2019.0,GE,57280.0,57433,1957-01-01,5710,True,A,27,C,5
217446,358802296,35880229600014,25.72Z,N,O,41,2019.0,ETI,57445.0,57566,1934-01-01,5710,True,A,25,C,3
217477,361200389,36120038900621,22.11Z,N,N,41,NaN,ETI,94150.0,94065,1961-01-01,5710,True,A,22,C,29


## Sauvegarde de la base SIREN finale

In [83]:
siren_df.to_csv("bdd/BaseSiren/siren_df.csv", index= False)